# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import re

import joblib
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine

nltk.download(['punkt', 'wordnet', 'stopwords', 'words',
               'averaged_perceptron_tagger', 'maxent_ne_chunker', 'omw'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\stefa\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-

True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)

X = df.message.values
Y = df.drop(['message', 'original', 'genre'], axis=1)
labels = Y.columns
Y = Y.values

### 2. Write a tokenization function to process your text data

In [4]:
stopwords_en = stopwords.words('english')

In [5]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN


def tokenize(text):

    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords_en]
    tokens = nltk.pos_tag(tokens)

    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok, tag in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok.strip(), pos=get_wordnet_pos(tag))
        clean_tokens.append(clean_tok)

    return clean_tokens


def display_results(y_pred, y_test):
    # display results
    cr = {}
    model_avg_f1 = np.empty(len(labels))
    for i, label in enumerate(labels):
        cr[label] = classification_report(
            y_test[:, i], y_pred[:, i], labels=df[label].unique(), zero_division=0)
        score = f1_score(y_test[:, i], y_pred[:, i], labels=df[label],
                         average='weighted', zero_division=0)
        model_avg_f1[i] = score

    model_avg_f1 = np.mean(model_avg_f1)
    print(f'The model weighted f1 score is {model_avg_f1}')
    return cr

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def create_pipeline(classifier, nr_jobs=1):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(classifier, n_jobs=nr_jobs))
    ])
    return pipeline

In [6]:
pipeline = create_pipeline(KNeighborsClassifier(n_jobs=-1))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000002B6A2367708>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='au

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict on test data
y_pred = pipeline.predict(X_test)

pipeline_cr = display_results(y_pred, y_test)
pipeline_cr.keys()

The model weighted f1 score is 0.9464108251187859


dict_keys(['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'])

In [9]:
print(pipeline_cr['medical_help'])

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6044
           1       0.50      0.00      0.00       510

    accuracy                           0.92      6554
   macro avg       0.71      0.50      0.48      6554
weighted avg       0.89      0.92      0.89      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
    'clf__estimator__leaf_size': [1, 5],
    'clf__estimator__n_neighbors': [6, 10, 15]
}

cv = GridSearchCV(create_pipeline(KNeighborsClassifier()),
                  param_grid=parameters, cv=2, verbose=5, n_jobs=-1)

In [14]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:  5.2min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  7.6min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# predict on test data
y_pred = cv.predict(X_test)
    
cv_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.945742522639369


In [16]:
cv.cv_results_

{'mean_fit_time': array([17.80915797, 17.78046679, 19.34417224, 17.31707835, 19.79517102,
        17.86345422]),
 'std_fit_time': array([0.10849679, 0.31099057, 1.76931262, 0.52496266, 1.16474009,
        0.64598072]),
 'mean_score_time': array([126.7625196 , 129.77820385, 128.49743271, 130.49579108,
        126.24271643, 126.25022852]),
 'std_score_time': array([3.10262561, 2.59397972, 2.77161956, 1.5138849 , 1.5054673 ,
        1.33795869]),
 'param_clf__estimator__leaf_size': masked_array(data=[1, 1, 1, 5, 5, 5],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_neighbors': masked_array(data=[6, 10, 15, 6, 10, 15],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__leaf_size': 1, 'clf__estimator__n_neighbors': 6},
  {'clf__estimator__leaf_size': 1, 'clf__estimator__n_neighbors': 10},
  {'clf__estimato

In [17]:
cv.best_params_

{'clf__estimator__leaf_size': 1, 'clf__estimator__n_neighbors': 6}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
model = create_pipeline(RandomForestClassifier(n_jobs=-1), -1)

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [19]:
y_pred = model.predict(X_test)

model_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9656898547107993


In [7]:
model = create_pipeline(RandomForestClassifier(n_jobs=-1), -1)

In [8]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000022711898D38>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [14]:
parameters = {
    'vect__ngram_range': [(1,1), (2,2)],
    'vect__min_df': [0.01, 0.05, 1],
    'vect__max_df': [0.8, 0.9, 1.0]
}

cv = GridSearchCV(create_pipeline(RandomForestClassifier()),
                  param_grid=parameters, cv=2, verbose=5, n_jobs=-1)

In [15]:
cv.fit(X_train, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 38.2min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [16]:
y_pred = cv.predict(X_test)

model_cr = display_results(y_pred, y_test)

The model weighted f1 score is 0.9644942842030704


In [17]:
cv.best_params_

{'vect__max_df': 1.0, 'vect__min_df': 1, 'vect__ngram_range': (1, 1)}

### 9. Export your model as a pickle file

In [20]:
joblib.dump(model, 'model.sav')

['model.sav']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.